# Info

# Inicialização

## Carregando dependências

In [0]:
# Utils
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
import os
import sys
sys.path.append(os.path.dirname(os.getcwd()))

# Modelo
from sklearn.model_selection import train_test_split
from lightgbm import LGBMClassifier
import lightgbm as lgb
from skopt import BayesSearchCV
from skopt.space import Real, Integer
from sklearn.metrics import accuracy_score, recall_score, confusion_matrix

## Funções

In [0]:
def cria_conjuntos(
    df: pd.DataFrame(),
):
    '''
    Info:
        Função que gera as bases de treino, validacao e teste de tal maneira que nao haja contas em mais de um conjunto.
    ----------
    Input:
        df: Conjunto de dados preprocessado.
    ----------
    Output:
        Conjuntos de treino, validacao e teste
    '''
    # Separando a base de dados em treino, validação e teste. 
    # A separação será feita baseada na quantidade de contas em cada conjunto
    # As contas em cada conjunto serão exclusivas (não haverá conta no conjunto de teste presente nos demais)

    # Substitundo valores nulos em offer_id
    df['offer_id'] = df['offer_id'].fillna('No offer')

    # Estratificando por oferta
    stratify_cols = ['offer_id']
    df_strat = (
        df[['account_id'] + stratify_cols]
        .drop_duplicates(subset='account_id')
    )

    # Obtendo as contas de treino e teste
    train_val_acc, test_acc, train_val_offer, _ = train_test_split(
        df_strat[['account_id'] + stratify_cols],
        df_strat['offer_id'],
        test_size=0.2,
        stratify=df_strat[stratify_cols],
        shuffle=True
    )

    # Obteando as contas de treino e validação
    train_acc, val_acc, _, _ = train_test_split(
        train_val_acc[['account_id'] + stratify_cols],
        train_val_offer,
        test_size=0.2,
        stratify=train_val_acc[stratify_cols],
        shuffle=True
    )

    # Criando as bases de dados
    df_train = df[df.account_id.isin(train_acc.account_id)]
    df_val = df[df.account_id.isin(val_acc.account_id)]
    df_test = df[df.account_id.isin(test_acc.account_id)]

    return df_train, df_val, df_test

def transform_data_for_model(
    df: pd.DataFrame(),
    offer: str,
    offers: list=None
):
    '''
    Info:
        Funcao utilizada para transformar um conjunto de dados para o modelo especifico.
    ----------
    Input:
        df: Dataset contendo os dados para serem transformados.
        offer: ID da oferta para transformar os dados para o modelo especifico.
    ----------
    Output:
        Dataset contendo os dados transformados
    '''
    # Removendo colunas de ID e as que são relacionadas com o ID da oferta
    rem_cols = [
        'offer_id', 'account_id',
        'discount_value',
        'min_value',
        'duration',
        'num_channels',
        'offer_type_bogo', 'offer_type_discount', 'offer_type_informational',
        'channel_email', 'channel_mobile', 'channel_social', 'channel_web'
    ]

    # Transformando o conjunto de dados para o modelo especifico
    if(offer != 'No offer'):
        return (
            df.assign(
                target = lambda x: np.where(x.offer_id == offer, 1, 0)
            )
            .drop(columns = rem_cols)
        )
    else:

        return (
            df.assign(
                target = lambda x: np.where(x.offer_id.isin(offers), 0, 1)
            )
            .drop(columns = rem_cols)
        )


def treinar_modelo_lgbm_bayes(
    df: pd.DataFrame(), 
    offer_id: str, 
    nome_arquivo: str
):
    '''
    Info:
        Treina um modelo LightGBM com otimização Bayesiana dos hiperparâmetros,
        avalia o desempenho na base de validação e plota a importância das features.
    ---------
    Input:
        df: dicionário com DataFrames divididos em treino e validação, por offer_id
        offer_id: chave usada para acessar os dados específicos de uma oferta
        nome_arquivo: nome do arquivo para salvar o modelo (não está sendo salvo no momento)
    ---------
    Output:
        melhor_modelo: modelo treinado com os melhores hiperparâmetros encontrados
    '''

    # Separa os dados de treino e validação, removendo a coluna 'target' das features
    X_train = df[offer_id]['train'].drop(columns=['target'])
    y_train = df[offer_id]['train']['target']
    X_val = df[offer_id]['validation'].drop(columns=['target'])
    y_val = df[offer_id]['validation']['target']

    # Espaço de busca para a otimização Bayesiana dos hiperparâmetros do LightGBM
    search_spaces = {
        'learning_rate': Real(0.01, 0.2, prior='log-uniform'),
        'num_leaves': Integer(20, 150),
        'max_depth': Integer(3, 15),
        'min_child_samples': Integer(10, 100),
        'subsample': Real(0.5, 1.0),
        'colsample_bytree': Real(0.5, 1.0),
        'reg_alpha': Real(0.0, 1.0),
        'reg_lambda': Real(0.0, 1.0)
    }

    # Inicializa o modelo LightGBM com alguns hiperparâmetros fixos
    lgbm = LGBMClassifier(
        objective='binary',
        random_state=42,
        class_weight='balanced',  # Para lidar com desbalanceamento da classe
        n_estimators=500,
        verbosity=-1  # Silencia os avisos
    )

    # Otimizador Bayesiano usando validação cruzada para encontrar os melhores hiperparâmetros
    opt = BayesSearchCV(
        estimator=lgbm,
        search_spaces=search_spaces,
        n_iter=30,            # Número de iterações da busca
        cv=3,                 # Validação cruzada 3-fold
        scoring='recall',     # Otimiza a sensibilidade (importante em casos de classe minoritária)
        n_jobs=-1,            # Usa todos os núcleos disponíveis
        verbose=0,
        random_state=42
    )

    # Executa a busca e treina o modelo com os melhores parâmetros
    opt.fit(X_train, y_train)

    # Recupera o melhor modelo encontrado
    melhor_modelo = opt.best_estimator_

    # Avalia o modelo na base de validação
    y_pred = melhor_modelo.predict(X_val)

    # Calcula métricas de avaliação
    acc = accuracy_score(y_val, y_pred)
    sens = recall_score(y_val, y_pred)
    cm = confusion_matrix(y_val, y_pred)
    tn, fp, fn, tp = cm.ravel() if cm.shape == (2, 2) else (0, 0, 0, 0)
    especificidade = tn / (tn + fp) if (tn + fp) > 0 else 0

    # Exibe os melhores hiperparâmetros encontrados
    print("\nMelhores Hiperparâmetros encontrados:")
    print(opt.best_params_)

    # Exibe as métricas de avaliação
    print(f"\nAcurácia:      {acc:.6f}")
    print(f"Sensibilidade: {sens:.6f}")
    print(f"Especificidade: {especificidade:.6f}")

    # Reajusta o modelo com todos os dados de treino
    melhor_modelo.fit(X_train, y_train)

    # Define o caminho padrão para salvar os modelos
    diretorio_modelos = "./"
    os.makedirs(diretorio_dbfs, exist_ok=True)

    caminho_completo = os.path.join(diretorio_modelos, f"{nome_arquivo}.pkl")

    # Salva o modelo
    with open(caminho_completo, "wb") as f:
        pickle.dump(melhor_modelo, f)

    print(f"\n Modelo salvo com sucesso em: {caminho_completo}")

    # Acessa o booster do modelo treinado
    booster = melhor_modelo.booster_

    # Plota a importância das features
    lgb.plot_importance(booster, max_num_features=20, importance_type='split', figsize=(10, 6))
    plt.title("Feature Importance (by split)")
    plt.show()

    # Retorna o modelo treinado com os melhores hiperparâmetros
    return melhor_modelo

def ensemble_voto_por_score(
    df_test: pd.DataFrame()
):
    '''
    Info:
        Carrega 9 modelos LightGBM do DBFS, faz predict_proba no conjunto de teste
        e escolhe, para cada linha, a oferta correspondente ao modelo com maior score.
    ----------
    Input:
        df_test: dataframe contendo dados de teste
    ----------
    Output:
        y_pred_df: DataFrame com scores dos 9 modelos e a oferta escolhida por linha
    '''

    # Caminho base onde os modelos estão salvos
    caminho_modelos = "./"

    # Modelos e suas respectivas ofertas (índice i = modelo{i+1})
    modelos_nomes = [f"modelo{i}.pkl" for i in range(1, 10)]
    
    ofertas = [
        '2298d6c36e964ae4a3e7e9706d1fb8c2',
        'ae264e3637204a6fb9bb56bc8210ddfd',
        '9b98b8c7a33c4b65b9aebfe6a799e6d9',
        'f19421c1d4aa40978ebb69ca19b0e20d',
        '2906b810c7d4411798c6938adc9daaa5',
        '0b1e1539f2cc45b7b9fa7c272da2e1d7',
        'fafdcd668e3743c1bb461111dcafc2a4',
        '4d5c57ea9a6940dd891ad53e9dbe8da0',
        'No offer'
    ]

    resultados = {}

    for nome_modelo, offer_id in zip(modelos_nomes, ofertas):
        caminho_modelo = os.path.join(caminho_modelos, nome_modelo)

        if not os.path.exists(caminho_modelo):
            raise FileNotFoundError(f"Modelo '{nome_modelo}' não encontrado em {caminho_modelo}.")

        # Carrega o modelo
        with open(caminho_modelo, "rb") as f:
            modelo = pickle.load(f)

        # Dados de teste
        X_test = df_test.drop(columns=['offer_id'])

        # Probabilidade da classe positiva
        y_prob = modelo.predict_proba(X_test)[:, 1]

        # Usa o nome do modelo como chave temporária
        resultados[nome_modelo] = y_prob

    # Concatena as colunas com os nomes temporários (modelo1.pkl, etc)
    y_pred_df = pd.DataFrame(resultados)

    # Mapear modelo -> oferta
    modelo_to_oferta = dict(zip(modelos_nomes, ofertas))

    # Identifica qual modelo teve o maior score por linha
    col_vencedora = y_pred_df.idxmax(axis=1)  # retorna modelo*.pkl

    # Mapeia para o nome da oferta
    y_pred_df['decisao_final'] = col_vencedora.map(modelo_to_oferta)

    df_final = (
        df_test
        .reset_index(drop=True)
        .merge(
            y_pred_df, 
            how = 'left', 
            left_index=True, 
            right_index=True
        )
    )

    acuracia_total = df_final.assign(acerto = lambda x: np.where(x.offer_id == x.decisao_final, 1, 0)).acerto.mean()

    print(f'Acurácia do modelo: {np.round(100*acuracia_total, 3)}%')

    return df_final

## Carregando dados

In [0]:
df = spark.read.format("json").load("ifood-case/data/processed/data_processed.json").toPandas()

# Modelagem

In [0]:
df.head()

In [0]:
# Quantidade de ofertas na base
offers = [c for c in list(df.offer_id.unique()) if c != None]
num_offers = len(offers)
num_offers

In [0]:
offers

## Gerando a base de treino, validacao e teste

In [0]:
df_train, df_val, df_test = cria_conjuntos(df=df)
display(df_train.head(2))
display(df_val.head(2))
display(df_test.head(2))

# Criando o sistema de classificação

A metologia proposta consiste de:
- Desenvolver um modelo de classificação para cada um dos 8 tipos de ofertas e um para detectar quando não tem oferta, aplicando a técnica One-vs-All, onde cada modelo vai ser treinado para identificar quando uma oferta em específico foi utilizada.
- Ao todo serão treinados 9 modelos de classificação, um para cada oferta em específico e um para detectar não ofertas.
- A oferta resultante da saída do sistema de classificação será aquela cujo score do modelo é o maior.

## Treinamento

In [0]:
# Criando um dicionario com os datasets tratados para cada modelo. Onde as chaves sao as ofertas.
models_datasets = {}
for offer in offers:
    models_datasets[offer] = {}

    # Obtendo os conjuntos de treino e validacao para cada modelo
    for dataset, df_ in [('train', df_train), ('validation', df_val)]:
        models_datasets[offer][dataset] = transform_data_for_model(df=df_, offer=offer)

# Adicionando um modelo espeficico para detectar nao ofertas
offer='No offer'
models_datasets[offer] = {}
for dataset, df_ in [('train', df_train), ('validation', df_val)]:
        models_datasets[offer][dataset] = transform_data_for_model(df=df_, offer=offer, offers=offers)

In [0]:
# Chaves relacionadas aos tipos de oferta
models_datasets.keys()

In [0]:
modelo1 = treinar_modelo_lgbm_bayes(models_datasets,'2298d6c36e964ae4a3e7e9706d1fb8c2','modelo1')

In [0]:
modelo2 = treinar_modelo_lgbm_bayes(models_datasets,'ae264e3637204a6fb9bb56bc8210ddfd','modelo2')

In [0]:
modelo3 = treinar_modelo_lgbm_bayes(models_datasets,'9b98b8c7a33c4b65b9aebfe6a799e6d9','modelo3')

In [0]:
modelo4 = treinar_modelo_lgbm_bayes(models_datasets,'f19421c1d4aa40978ebb69ca19b0e20d','modelo4')

In [0]:
modelo5 = treinar_modelo_lgbm_bayes(models_datasets,'2906b810c7d4411798c6938adc9daaa5','modelo5')

In [0]:
modelo6 = treinar_modelo_lgbm_bayes(models_datasets,'0b1e1539f2cc45b7b9fa7c272da2e1d7','modelo6')

In [0]:
modelo7 = treinar_modelo_lgbm_bayes(models_datasets,'fafdcd668e3743c1bb461111dcafc2a4','modelo7')

In [0]:
modelo8 = treinar_modelo_lgbm_bayes(models_datasets,'4d5c57ea9a6940dd891ad53e9dbe8da0','modelo8')

In [0]:
modelo9 = treinar_modelo_lgbm_bayes(models_datasets,'No offer','modelo9')

## Avaliação 

In [0]:
# Removendo colunas de ID e as que são relacionadas com o ID da oferta
rem_cols = [
    'account_id',
    'discount_value',
    'min_value',
    'duration',
    'num_channels',
    'offer_type_bogo', 'offer_type_discount', 'offer_type_informational',
    'channel_email', 'channel_mobile', 'channel_social', 'channel_web'
]
            
df_test = df_test.drop(columns = rem_cols)

In [0]:
df_resultados = ensemble_voto_por_score(df_test)
df_resultados.head()

In [0]:
amt_com_cupom = np.round(df_resultados[(df_resultados.decisao_final == df_resultados.offer_id) & (df_resultados.offer_id != 'No offer')].amount.mean(), 2)
amt_sem_cupom = np.round(df_resultados[(df_resultados.decisao_final == df_resultados.offer_id) & (df_resultados.offer_id == 'No offer')].amount.mean(), 2)
print(f"Media do amount quando o usuario usa cupom: R${amt_com_cupom}")
print(f"Media do amount quando o usuario nao usa cupom: R${amt_sem_cupom}")
print(f'Usuários que usam cupom gastam, em média, {np.round(100*((amt_com_cupom/amt_sem_cupom) - 1), 2)}% a mais')

In [0]:
# Ganho em reais por ter aplicado o cupom efetivamente (considerando que o amount de compras sem cupom são, em média, 60% menores)
amt_total_com_cupom = df_resultados[(df_resultados.decisao_final == df_resultados.offer_id) & (df_resultados.offer_id != 'No offer')].amount.sum()

amt_total_sem_cupom = df_resultados[(df_resultados.decisao_final == df_resultados.offer_id) & (df_resultados.offer_id != 'No offer')].amount.sum() * 0.4

print(f'Ganho monetário na aplicação de cupons segundo o sistema: R${np.round(amt_total_com_cupom - amt_total_sem_cupom, 2)}')

## Salvando os resultados

In [0]:
df_resultados.to_csv("ifood-case/data/processed/3_results_model.csv")